# 清理並安裝 PyTorch

In [ ]:
import os
import sys

os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
os.environ['FORCE_CUDA'] = '1'

!pip uninstall -y torch torchvision torchaudio numpy > /dev/null 2>&1
!pip install --upgrade --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 > /dev/null 2>&1

os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')
sys.exit(0)

SystemExit: 0

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 安裝 NumPy 兼容性版本

In [ ]:
import sys

!pip install --upgrade --force-reinstall --no-cache-dir numpy==1.26.4 > /dev/null 2>&1
sys.exit(0)

SystemExit: 0

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 安裝其餘所有必要的 Python 庫（transformers, diffusers, gradio, wordcloud, matplotlib, Pillow）

In [ ]:
import os
import sys
import time
import torch

!pip install --upgrade --no-cache-dir gradio wordcloud matplotlib Pillow > /dev/null 2>&1
!pip install --upgrade --no-cache-dir huggingface_hub > /dev/null 2>&1
!pip install --upgrade --no-cache-dir diffusers transformers accelerate safetensors > /dev/null 2>&1
sys.exit(0)

SystemExit: 0

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 包含所有應用邏輯（模型加載、圖片生成、回饋處理、關鍵字雲生成、Gradio 介面）

In [ ]:
import os
import sys
import time
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud
from PIL import Image
import io
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from transformers import AutoTokenizer

model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16 if device == "cuda" else torch.float32)
pipe.to(device)

feedback_keywords = []
print(f"DEBUG: Initial feedback_keywords: {feedback_keywords}")

def clear_feedback_keywords():
    global feedback_keywords
    feedback_keywords.clear()
    print(f"DEBUG: clear_feedback_keywords called. Current feedback_keywords: {feedback_keywords}")
    return generate_keyword_cloud()

def get_prompts_from_feedback_keywords():
    positive_keywords = []
    negative_keywords = []
    for keyword in feedback_keywords:
        if keyword.startswith("Not_"):
            negative_keywords.append(keyword[4:].replace("_", " "))
        else:
            positive_keywords.append(keyword)
    print(f"DEBUG: get_prompts_from_feedback_keywords. Positive: '{', '.join(positive_keywords)}', Negative: '{', '.join(negative_keywords)}'")
    return ", ".join(positive_keywords), ", ".join(negative_keywords)

def generate_image_with_diffusers(input_image_path, style_prompt, additional_prompt):
    if input_image_path is None:
        return "請上傳一張圖片。"

    positive_feedback_str, negative_feedback_str = get_prompts_from_feedback_keywords()

    full_prompt = f"{style_prompt} style, {additional_prompt}" if additional_prompt else f"{style_prompt} style"
    if positive_feedback_str:
        full_prompt = f"{full_prompt}, {positive_feedback_str}"

    negative_prompt = negative_feedback_str if negative_feedback_str else ""

    print(f"DEBUG: generate_image_with_diffusers. Full Prompt: '{full_prompt}', Negative Prompt: '{negative_prompt}'")

    init_image = Image.open(input_image_path).convert("RGB")
    generated_image = None
    generator = torch.Generator(device=device).manual_seed(0)
    generated_image = pipe(
        prompt=full_prompt,
        negative_prompt=negative_prompt,
        image=init_image,
        strength=0.75,
        guidance_scale=7.5,
        generator=generator
    ).images[0]

    return generated_image

def generate_keyword_cloud():
    print(f"DEBUG: generate_keyword_cloud called. Current feedback_keywords: {feedback_keywords}")
    if not feedback_keywords:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.text(0.5, 0.5, "No feedback data to generate keyword cloud", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=16)
        ax.axis('off')
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        plt.close(fig)
        buf.seek(0)
        return Image.open(buf)

    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        collocations=False,
    ).generate(' '.join(feedback_keywords))

    img_buf = io.BytesIO()
    wordcloud.to_image().save(img_buf, format='PNG')
    img_buf.seek(0)

    return Image.open(img_buf)

def handle_feedback(output_image, selected_style, feedback_choice, additional_prompt):
    print(f"DEBUG: handle_feedback called.")
    print(f"DEBUG: Input - selected_style: '{selected_style}', feedback_choice: '{feedback_choice}', additional_prompt: '{additional_prompt}'")

    words_from_prompt = [word.strip() for word in additional_prompt.split(',') if word.strip()]

    if feedback_choice == "👍 Like":
        feedback_keywords.append(selected_style)
        feedback_keywords.extend(words_from_prompt)
    elif feedback_choice == "👎 Dislike":
        feedback_keywords.append("Not_" + selected_style.replace(" ", "_"))
        feedback_keywords.extend(["Not_" + word.replace(" ", "_") for word in words_from_prompt])
    else:
        print(f"DEBUG: No valid feedback_choice selected or mismatch: '{feedback_choice}'")

    print(f"DEBUG: After handle_feedback, feedback_keywords: {feedback_keywords}")
    return generate_keyword_cloud()


def gradio_interface():
    available_styles = ["Anime", "Realistic", "2D Illustration", "Oil Painting", "Watercolor", "Cyberpunk"]

    with gr.Blocks(css="#feedback_row { margin-top: 20px; }") as demo:
        gr.Markdown(
            """
            # 🎨 AI Image Style Transfer Platform (Diffusers Version) 🎨
            Upload your image, choose a style, and enter a description to generate a new styled image!
            Please provide feedback on the generated images to help us improve the recommendation system.
            """
        )

        with gr.Row():
            with gr.Column(scale=1):
                image_input = gr.Image(type="filepath", label="Upload Image")
                style_selection = gr.Dropdown(
                    choices=available_styles,
                    value="Realistic",
                    label="Select Style",
                    interactive=True
                )
                additional_prompt_input = gr.Textbox(
                    label="Additional Prompt (e.g., black dress, mountain top)",
                    placeholder="Enter additional description to influence the generated result..."
                )
                generate_button = gr.Button("🚀 Generate Image")

            with gr.Column(scale=1):
                output_image = gr.Image(label="Generated Result", type="pil", interactive=False)

                applied_positive_keywords_display = gr.Textbox(label="Applied Positive Keywords", interactive=False)
                applied_negative_keywords_display = gr.Textbox(label="Applied Negative Keywords", interactive=False)


                with gr.Row(elem_id="feedback_row"):
                    feedback_radio = gr.Radio(
                        ["👍 Like", "👎 Dislike"],
                        label="Your Feedback?",
                        value=None
                    )
                    submit_feedback_button = gr.Button("Submit Feedback")
                    reset_preferences_button = gr.Button("🗑️ Reset Preferences")

                with gr.Row():
                    keyword_cloud_output = gr.Image(label="Keyword Cloud Recommendation", interactive=False)

                generate_button.click(
                    fn=get_prompts_from_feedback_keywords,
                    inputs=[],
                    outputs=[applied_positive_keywords_display, applied_negative_keywords_display]
                ).success(
                    fn=generate_image_with_diffusers,
                    inputs=[image_input, style_selection, additional_prompt_input],
                    outputs=[output_image]
                )

                submit_feedback_button.click(
                    fn=handle_feedback,
                    inputs=[
                        output_image,
                        style_selection,
                        feedback_radio,
                        additional_prompt_input
                    ],
                    outputs=keyword_cloud_output
                ).success(
                    fn=get_prompts_from_feedback_keywords,
                    inputs=[],
                    outputs=[applied_positive_keywords_display, applied_negative_keywords_display]
                )

                reset_preferences_button.click(
                    fn=clear_feedback_keywords,
                    inputs=[],
                    outputs=keyword_cloud_output
                ).success(
                    fn=lambda: ("", ""),
                    inputs=[],
                    outputs=[applied_positive_keywords_display, applied_negative_keywords_display]
                )

                demo.load(
                    fn=lambda: (generate_keyword_cloud(), "", ""),
                    inputs=[],
                    outputs=[keyword_cloud_output, applied_positive_keywords_display, applied_negative_keywords_display]
                )

            demo.launch(debug=True, share=True)

gradio_interface()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

DEBUG: Initial feedback_keywords: []
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://913e108013e23aa672.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG: generate_keyword_cloud called. Current feedback_keywords: []
DEBUG: generate_keyword_cloud called. Current feedback_keywords: []
DEBUG: get_prompts_from_feedback_keywords. Positive: '', Negative: ''
DEBUG: get_prompts_from_feedback_keywords. Positive: '', Negative: ''
DEBUG: generate_image_with_diffusers. Full Prompt: 'Realistic style', Negative Prompt: ''


  0%|          | 0/37 [00:00<?, ?it/s]

DEBUG: get_prompts_from_feedback_keywords. Positive: '', Negative: ''
DEBUG: get_prompts_from_feedback_keywords. Positive: '', Negative: ''
DEBUG: generate_image_with_diffusers. Full Prompt: 'Anime style', Negative Prompt: ''


  0%|          | 0/37 [00:00<?, ?it/s]